In [1]:
import os
import soundfile
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten

In [2]:
# Function to extract MFCC features from audio files
def extract_features(file_path, mfcc=True, chroma=True, mel=True):
    X, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
    if chroma:
        stft = np.abs(librosa.stft(X))
    result = np.array([])
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, chroma))
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
        result = np.hstack((result, mel))
    return result

In [3]:
# Directory containing audio files
train_dir = 'Audios/train'
test_dir = 'Audios/test'

In [4]:
# List of emotions (folder names)
emotions = os.listdir(train_dir)

In [5]:
print(emotions)

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [6]:
# Dictionary to map emotions to labels
emotion_to_label = {emotion: idx for idx, emotion in enumerate(emotions)}

In [7]:
# Lists to store features and labels
X_train = []
y_train = []

In [8]:
# Extract features and labels from train audio files
for emotion in emotions:
    files = os.listdir(os.path.join(train_dir, emotion))
    for file in files:
        file_path = os.path.join(train_dir, emotion, file)
        feature = extract_features(file_path)
        X_train.append(feature)
        y_train.append(emotion_to_label[emotion])

In [9]:
# Convert lists to numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

In [10]:
# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))  # Added layer
model.add(Dropout(0.5))  # Added dropout
model.add(Dense(128, activation='relu'))  # Added layer
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))  # Added layer
model.add(Dropout(0.5))  # Added dropout
model.add(Dense(len(emotions), activation='softmax'))

In [11]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
# Train the model
model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.2)

Epoch 1/500
70/70 [==============================] - 0s 5ms/step - loss: 13.5355 - accuracy: 0.1795 - val_loss: 2.2394 - val_accuracy: 0.0089
Epoch 2/500
70/70 [==============================] - 0s 3ms/step - loss: 3.0480 - accuracy: 0.1799 - val_loss: 2.0474 - val_accuracy: 0.0000e+00
Epoch 3/500
70/70 [==============================] - 0s 3ms/step - loss: 2.1949 - accuracy: 0.1826 - val_loss: 2.1069 - val_accuracy: 0.0000e+00
Epoch 4/500
70/70 [==============================] - 0s 3ms/step - loss: 2.0032 - accuracy: 0.1732 - val_loss: 2.1929 - val_accuracy: 0.0000e+00
Epoch 5/500
70/70 [==============================] - 0s 3ms/step - loss: 1.9565 - accuracy: 0.1737 - val_loss: 2.2826 - val_accuracy: 0.0000e+00
Epoch 6/500
70/70 [==============================] - 0s 3ms/step - loss: 1.8852 - accuracy: 0.1996 - val_loss: 2.3835 - val_accuracy: 0.0000e+00
Epoch 7/500
70/70 [==============================] - 0s 3ms/step - loss: 1.8802 - accuracy: 0.1741 - val_loss: 2.5005 - val_accuracy:

In [13]:
# Save the model
model.save('audio_emotion_model5.h5')